In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.get_dataframe()


# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.



In [3]:
df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df

print(len(df.CUSTOMER.unique()))


NameError: name 'ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df' is not defined

In [0]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# find all customer_names that have other customer names that start with this customer name

#idx_match = 0
#idx = 0
#unique_customer_names = df['CUSTOMER'].unique()

#for n in unique_customer_names:
    
#    df_f = df[(df['CUSTOMER'].str.startswith(n, na=False))&(df['CUSTOMER']!=n)]
#    match_list = df_f['CUSTOMER'].unique()
#    if len(match_list)>0:
#        print(n, match_list)
#        print(idx, idx_match)
#        print()
#        idx_match+=1
#    else:
#        idx+=1

In [0]:
ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df # For this sample code, simply copy input to output


# Write recipe outputs
ACCOUNTS_WITH_POTENTIAL_DUPLICATES = dataiku.Dataset("ACCOUNTS_WITH_POTENTIAL_DUPLICATES")
ACCOUNTS_WITH_POTENTIAL_DUPLICATES.write_with_schema(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df)